# Hyperparameter optimization for machine learning

Many machine learning models have hyperparamters -- parameters that control some aspect of the model. The exact setting of these hyperparameters can dramatically impact the performance of your underlying model. Fortunately, most hyperparameters can be tried in parallel, making the task of *hyperparemter optimization* a great fit for PyWren. 

Here we use a simple dataset included in scikit-learn to show how to do hyperparameter optimization across a number of different datasets, and a number of different cross-validations 

In [ ]:
%pylab inline
import pywren
import sklearn
import seaborn as sns
import itertools
import pandas as pd
from sklearn.model_selection import train_test_split
import sklearn.svm

from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline, Pipeline

import warnings
warnings.filterwarnings("ignore")

# get the data

First we load in the data from scikit learn and examine it. Here we will be using an existing dataset of breast cancer tumor properties that's shipped with scikit-learn. This is a small binary classification problem, and the hyperparameter optimization we are doing here is reletively trivial. Hopefully this will inspire good ideas. 

In [ ]:
# load the dataset from scikit learn
from sklearn.datasets import load_breast_cancer
bc = load_breast_cancer()

Let's explore properties of the dataset. 

In [ ]:
print(bc['DESCR'])

The data is in the `data` key and the classification label is in the `target` key. Load those:

In [ ]:
# load the data as X and y
X = bc['data']
y = bc['target']

First we explore generic prediction on the dataset with an off-the-shelf classifier. We use a scikit-learn pipeline which does the following:

1. standardize each input feature to zero-mean unit variance 
2. Apply a SVM with a given set of hyperparameters

In [ ]:

# create a train/test split for cross-validation
X_train, X_test, y_train, y_test = train_test_split(
     X, y, test_size=0.2, random_state=0)

# construct a scikit learn pipeline which normalizes
# the data and then runs a SVM 

p = Pipeline(steps=[('standardize', StandardScaler()), 
                    ('learn', sklearn.svm.SVC(C=0.1, kernel='poly', degree=3))])


# fit the data
p.fit(X_train, y_train)


Then we can evaluate the behavior of the classifier on the held-out set

In [ ]:

# predict the held-out data
pred = p.predict(X_test)

# measure the accuracy
sklearn.metrics.accuracy_score(y_test, pred)

# use PyWren to try many HP values

Now we try for many folds and many hyperparameters with pywren. We can also try many different folds to get confidence intervals on the sensitivyt of our new estimator. 


In [ ]:
# now we can train lots of models! and get error bars! 

def train_model(args):
    """
    
    """
    # extract the args
    (X_train, X_test, y_train, y_test), hps = args
    
    import sklearn.svm 
    
    # create the pipeline
    p = Pipeline(steps=[('standardize', StandardScaler()), 
                        ('learn', sklearn.svm.SVC(**hps))])
    
    # fit
    p.fit(X_train, y_train)
    return p


wrenexec = pywren.default_executor()


FOLD_N = 10 # number of cross-validation sets

cv_sets = []
for i in range(FOLD_N):
    cv_sets.append(train_test_split(
                     X, y, test_size=0.4, random_state=i))

# set of SVM hyperparameters we want to try
hps =  [dict(C=10.0, kernel='rbf', degree=3), 
        dict(C=1.0, kernel='rbf', degree=3),
        dict(C=0.1, kernel='rbf', degree=3)]


# combine them all using itertools
args = list(itertools.product(cv_sets, hps) )


In [ ]:
# perform the map                   
futures = wrenexec.map(train_model, args)
results = pywren.get_all_results(futures)

# Analyzing the results

We construct a dataframe of the results to facilitate subsequent analysis

In [ ]:
# construct a dataframe using the resulting models to predict accuracy on their held-out dataset
res = []
for m, ((X_train, X_test, y_train, y_test), hps) in zip(results, args):
    pred = m.predict(X_test)
    acc = sklearn.metrics.accuracy_score(y_test, pred)
    hp_row = hps.copy()
    hp_row['accuracy'] = acc
    res.append(hp_row)
df = pd.DataFrame(res)

### we use seaborn to plot the results

In [ ]:
sns.boxplot(x='C', y='accuracy', data=df)
sns.stripplot(x='C', y='accuracy', data=df, color='k', jitter=True)


# Exercise: Try different classifiers

Many different classifiers can work for the simple binary classification problem. We encourage you to try several from scikit learn below. Perhaps logistic regression or a random forest? 

In [ ]:
# now we can train lots of models! and get error bars! 

def train_model(args):
    """
    
    """
    # extract the args
    (X_train, X_test, y_train, y_test), (classifier_name, hps) = args
    
    import sklearn.svm 
    import sklearn.ensemble
    
    # create the pipeline
    if classifier_name == "svm":
        p = Pipeline(steps=[('standardize', StandardScaler()), 
                            ('learn', sklearn.svm.SVC(**hps))])
    elif classifier_name == "myclassifierPipeline":
        # TODO CREATE YOUR PIPELINE HERE
        #
    ## TODO ADD MORE 
    
    # fit
    p.fit(X_train, y_train)
    return p


wrenexec = pywren.default_executor()


FOLD_N = 4 # number of cross-validation sets

cv_sets = []
for i in range(FOLD_N):
    cv_sets.append(train_test_split(
                     X, y, test_size=0.4, random_state=i))

# set of SVM hyperparameters we want to try
models_and_hps =  [('svm', dict(C=10.0, kernel='rbf', degree=3)), 
                   ('svm', dict(C=1.0, kernel='rbf', degree=3)),
                   ('svm', dict(C=0.1, kernel='rbf', degree=3)),
                   # Add additional configurations and 
                  ]


# combine them all using itertools
args = list(itertools.product(cv_sets, models_and_hps) )


In [ ]:
# perform the map  
## TODO call pywren and get the results
import sklearn.ensemble.forest
futures = wrenexec.map(train_model, args)
results = pywren.get_all_results(futures)

In [ ]:
# construct a dataframe using the resulting models to predict accuracy on their held-out dataset
res = []
for m, ((X_train, X_test, y_train, y_test), (model_name, hps)) in zip(results, args):
    pred = m.predict(X_test)
    acc = sklearn.metrics.accuracy_score(y_test, pred)
    hp_row = hps.copy()
    hp_row['accuracy'] = acc
    hp_row['hp_str'] = str(hps)
    hp_row['model_name'] = model_name
    res.append(hp_row)
df = pd.DataFrame(res)

In [ ]:
sns.boxplot(x='model_name', y='accuracy', data=df, hue="hp_str")
#sns.stripplot(x='C', y='accuracy', data=df, color='k', jitter=True)


# additional things to try

Some additional recommended exercises:
1. Scikit-learn has several other datasets you might try?
2. are there other metrics such as `ROC` that might be a better indicator of performance? 
